In [ ]:
import gc
from glob import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time

import xgboost as xgb
from sklearn.model_selection import StratifiedKFold
from sklearn import metrics

import optuna
from optuna.visualization import (
    plot_contour
    , plot_edf
    , plot_intermediate_values
    , plot_optimization_history
    , plot_parallel_coordinate
    , plot_param_importances
    , plot_slice
)

import sys
sys.path.append("../utils")
from metrics import compute_recall_at4, compute_normalized_gini, compute_amex_metric

np.random.seed(2112)
pd.set_option('display.max_columns', None)

***
## load and prepare data

In [ ]:
!ls ../data/processed/dsv04

In [ ]:
train = pd.read_parquet("../data/processed/dsv04/train.parquet")
train_labels = pd.read_csv("../data/raw/train_labels.csv", index_col="customer_ID")

In [ ]:
input_feats = train.columns.tolist()
len(input_feats)

In [ ]:
train = pd.merge(train, train_labels, how="inner", left_index=True, right_index=True)
train = train.reset_index()

del train_labels
gc.collect()

***
## model tuning


In [ ]:
skf = StratifiedKFold(n_splits=3, random_state=2112, shuffle=True)
skf_split = list(skf.split(train, train["target"].values))

In [ ]:
default_params = {
    # general parameters
    'booster':'gbtree',
    'objective':'binary:logistic',
    'seed':2112,
    'eta': 0.05,
    'tree_method':'hist',
    'grow_policy':'depthwise',
}

In [ ]:
def train_models(dataframe: pd.DataFrame, split: list, model_params: dict) -> pd.DataFrame:
        
    # dataframe to store the oof predictions
    oof = dataframe[["target"]].copy()
    oof["pred"] = -1
    MAX_ITERATIONS = model_params.pop("iterations")

    for train_idx,valid_idx in split:
        
        train_df = dataframe.loc[train_idx,:]
        valid_df = dataframe.loc[valid_idx,:]
                                        
        train_dset = xgb.DMatrix(
            data=train_df.loc[:,input_feats],
            label=train_df.loc[:,"target"].values,
        )
        valid_dset = xgb.DMatrix(
            data=valid_df.loc[:,input_feats],
            label=valid_df.loc[:,"target"].values,
        )
        model = xgb.train(
            params = model_params,
            dtrain=train_dset,
            num_boost_round=MAX_ITERATIONS,
        )        
        oof.loc[valid_df.index,"pred"] = model.predict(valid_dset)
        
        del train_df,valid_df,train_dset,valid_dset,model
        gc.collect()
    
    return oof

In [ ]:
def objective(trial):
    sampled_params = {
        # general booster config
        "max_bin" : 2**trial.suggest_int("max_bin_exp", 6, 10) - 1,
        "max_depth" : trial.suggest_int("max_depth", 4, 10),
        "iterations" : trial.suggest_int("iterations", 1000, 5000, 50),
        # regularization
        "colsample_bytree" : trial.suggest_discrete_uniform("colsample_bytree", 0.1, 0.4, 0.05),
        "subsample" : trial.suggest_discrete_uniform("subsample", 0.8, 1.0, 0.05),
        "alpha" : trial.suggest_float("alpha", 0., 100.),
        "lambda" : trial.suggest_float("lambda", 0., 10.),
        "min_child_weight" : trial.suggest_int("min_child_weight", 100, 3000, 100),
        "min_split_loss" : trial.suggest_loguniform("min_split_loss", 1e-10, 1e0),
        "max_delta_step" : trial.suggest_loguniform("max_delta_step", 1e-10, 1e1),
    }    
    model_params = {**default_params, **sampled_params}
    
    oof = train_models(train, skf_split, model_params)
    metric = compute_amex_metric(oof.target.values, oof.pred.values)
    return metric

In [ ]:
do_optimize = True

study = optuna.create_study(
    study_name="xgboost-gbtree-dsv04",
    direction='maximize',
    storage='sqlite:///xgboost-gbtree-dsv04.db',
    load_if_exists=True,
)

if do_optimize:
    study.optimize(
        objective, 
        n_trials=1000, 
        timeout=172800, # 2-days
        n_jobs=1, 
        gc_after_trial=True,
    ) 

In [ ]:
study.trials_dataframe().sort_values("value", ascending=False).head(20)

In [ ]:
plot_optimization_history(study)

In [ ]:
plot_param_importances(study)

In [ ]:
plot_slice(study)

In [ ]:
plot_edf(study)

In [ ]:
plot_parallel_coordinate(study)

In [ ]:
best_params = dict(study.best_params)
best_params["max_bin"] = 2**best_params.pop("max_bin_exp")-1
best_params = {**default_params, **best_params}
best_params

***